In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'png'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/home/hicham/Work/blog/posts/sharding-with-jax':
  os.chdir(r'/home/hicham/Work/blog/posts/sharding-with-jax')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


In [2]:
from jax.random import PRNGKey
import jax.numpy as jnp

import keras_core as K
import datetime


N_TRAIN = 8*2**(10+4)
N_EVAL = 8*2**10

key = PRNGKey(0)

(x_train, y_train), (x_test, y_test) = K.datasets.mnist.load_data()
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = jnp.expand_dims(x_train, -1)
x_test = jnp.expand_dims(x_test, -1)

input_shape = 28, 28, 1

model = K.Sequential(
  [
    K.layers.Flatten(input_shape),
    K.layers.Dense(128, activation="relu"),
    K.layers.Dense(128, activation="relu"),
    K.layers.Dropout(rate=.5),
    K.layers.Dense(128, activation="relu"),
    K.layers.Dense(128, activation="relu"),
    K.layers.Dropout(rate=.5),
    K.layers.Dense(128, activation="relu"),
    K.layers.Dense(128, activation="relu"),
    K.layers.Dropout(rate=.5),
    K.layers.Dense(128, activation="relu"),
    K.layers.Dense(10, activation="softmax"),
  ]
)

model.summary()

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=K.optimizers.Adam(0.001),
    metrics=["accuracy"]
)


In [3]:
baseline_logs = "logs/baseline/" + datetime.now().strftime("%Y%m%d-%H%M%S")

baseline_callback = K.callbacks.TensorBoard(
  log_dir = baseline_logs,
  histogram_freq = 1,
  profile_batch = '500,520'
)


model.fit(
  x_train,
  y_train,
  epochs=2,
  validation_split=0.20,
  callbacks = [baseline_callback])